In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import folium
import time
import os
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [114]:
import os
os.environ['OGR_GEOMETRY_ACCEPT_UNCLOSED_RING'] = 'NO'

train_stations = gpd.read_file("./RapidTransitSystemStation.shp")

train_stations_4326 = train_stations.to_crs(epsg = 4326)


train_stations_4326['geometry'] = train_stations_4326['geometry'].apply(
    lambda geom: geom.centroid if geom.geom_type in ['Polygon', 'MultiPolygon'] else geom
)

train_stations_4326

,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
0,0,None,None,MRT,GALI BATU DEPOT,POINT (103.75609 1.39759)
1,0,None,None,MRT,HILLVIEW MRT STATION,POINT (103.76743 1.36234)
2,0,None,None,MRT,BEAUTY WORLD MRT STATION,POINT (103.77582 1.3412)
3,0,None,None,MRT,HUME MRT STATION,POINT (103.7691 1.35451)
4,0,None,None,MRT,BUKIT PANJANG MRT STATION,POINT (103.76157 1.37916)
...,...,...,...,...,...,...
225,0,None,None,MRT,KATONG PARK MRT STATION,POINT (103.88618 1.29782)
226,0,None,None,MRT,MARINE TERRACE MRT STATION,POINT (103.91532 1.30679)
227,0,None,None,MRT,TANJONG RHU MRT STATION,POINT (103.87345 1.29721)
228,0,None,None,MRT,MARINE PARADE MRT STATION,POINT (103.90551 1.30287)


In [119]:
train_stations_3857 = train_stations.to_crs(3857)

train_stations_3857["geometry"] = train_stations_3857["geometry"].centroid

train_stations_3857

,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
0,0,None,None,MRT,GALI BATU DEPOT,POINT (11550075.331 155593.891)
1,0,None,None,MRT,HILLVIEW MRT STATION,POINT (11551337.385 151669.297)
2,0,None,None,MRT,BEAUTY WORLD MRT STATION,POINT (11552271.325 149315.735)
3,0,None,None,MRT,HUME MRT STATION,POINT (11551523.809 150797.503)
4,0,None,None,MRT,BUKIT PANJANG MRT STATION,POINT (11550685.441 153542.239)
...,...,...,...,...,...,...
225,0,None,None,MRT,KATONG PARK MRT STATION,POINT (11564556.889 144485.295)
226,0,None,None,MRT,MARINE TERRACE MRT STATION,POINT (11567800.045 145483.386)
227,0,None,None,MRT,TANJONG RHU MRT STATION,POINT (11563139.751 144417.599)
228,0,None,None,MRT,MARINE PARADE MRT STATION,POINT (11566708.269 145046.784)


In [120]:
mrt_file_loc = "C:/Users/kaili/OneDrive/Documents/DSA4264/mrt_stations/mrt_stations.shp"
mrt_gdf = gpd.read_file(mrt_file_loc)

In [121]:
mrt_gdf["lat_lng"] = gpd.points_from_xy(mrt_gdf["LATITUDE"], mrt_gdf["LONGITUDE"])

mrt_gdf["line"] = mrt_gdf["CODE"].str.slice(0, 2) 

mrt_lines_code = ["NS", "EW", "CC", "NE", "TE", "DT"]

mrt_gdf_wo_lrt = mrt_gdf[mrt_gdf["line"].isin(mrt_lines_code)].reset_index()
mrt_gdf_wo_lrt = mrt_gdf_wo_lrt.drop("index", axis = 1)

mrt_gdf_wo_lrt["STN_NAM_DE"] = mrt_gdf_wo_lrt["BUILDING"].str.split('(').str[0].str.strip()

In [122]:
mrt_gdf_wo_lrt2 = mrt_gdf_wo_lrt[["CODE", "STN_NAM_DE", "LINE", "line", "COLOR"]]

mrt_gdf_wo_lrt2["Order"] = pd.to_numeric(mrt_gdf_wo_lrt2["CODE"].str.slice(2, None))

mrt_gdf_wo_lrt2[mrt_gdf_wo_lrt["line"] == "TE"].sort_values("Order", ascending = False)

,CODE,STN_NAM_DE,LINE,line,COLOR,Order
157,TE22,GARDENS BY THE BAY MRT STATION,Thomson-East Coast Line,TE,Brown,22
156,TE20,MARINA BAY MRT STATION,Thomson-East Coast Line,TE,Brown,20
155,TE19,SHENTON WAY MRT STATION,Thomson-East Coast Line,TE,Brown,19
154,TE18,MAXWELL MRT STATION,Thomson-East Coast Line,TE,Brown,18
153,TE17,OUTRAM PARK MRT STATION,Thomson-East Coast Line,TE,Brown,17
152,TE16,HAVELOCK MRT STATION,Thomson-East Coast Line,TE,Brown,16
151,TE15,GREAT WORLD MRT STATION,Thomson-East Coast Line,TE,Brown,15
150,TE14,ORCHARD MRT STATION,Thomson-East Coast Line,TE,Brown,14
149,TE13,ORCHARD BOULEVARD MRT STATION,Thomson-East Coast Line,TE,Brown,13
148,TE12,NAPIER MRT STATION,Thomson-East Coast Line,TE,Brown,12


In [123]:
new_brown_line = ['TANJONG RHU MRT STATION','KATONG PARK MRT STATION', 'TANJONG KATONG MRT STATION', 
                  'MARINE PARADE MRT STATION', 'MARINE TERRACE MRT STATION', 'SIGLAP MRT STATION', 'BAYSHORE MRT STATION']

In [124]:
n = len(new_brown_line)
add_tel_stations = []
for i in range(n):
    no = i + 23
    station_name = new_brown_line[i]
    add_tel_stations.append({
        "CODE": "TE" + str(no),
        "STN_NAM_DE": station_name,
        "LINE": "Thomson-East Coast Line",
        "line": "TE",
        "COLOR": "Brown",
        "Order": no
    })

new_tel_stations = pd.DataFrame(add_tel_stations)

mrt_gdf_wo_lrt2 = pd.concat([mrt_gdf_wo_lrt2, new_tel_stations], axis = 0)
mrt_gdf_wo_lrt2

,CODE,STN_NAM_DE,LINE,line,COLOR,Order
0,NS1,JURONG EAST MRT STATION,North-South Line,NS,Red,1
1,NS2,BUKIT BATOK MRT STATION,North-South Line,NS,Red,2
2,NS3,BUKIT GOMBAK MRT STATION,North-South Line,NS,Red,3
3,NS4,CHOA CHU KANG MRT STATION,North-South Line,NS,Red,4
4,NS5,YEW TEE MRT STATION,North-South Line,NS,Red,5
...,...,...,...,...,...,...
2,TE25,TANJONG KATONG MRT STATION,Thomson-East Coast Line,TE,Brown,25
3,TE26,MARINE PARADE MRT STATION,Thomson-East Coast Line,TE,Brown,26
4,TE27,MARINE TERRACE MRT STATION,Thomson-East Coast Line,TE,Brown,27
5,TE28,SIGLAP MRT STATION,Thomson-East Coast Line,TE,Brown,28


In [125]:
mrt_stations_3857 = mrt_gdf_wo_lrt2.merge(train_stations_3857, how = "left", on = "STN_NAM_DE")
mrt_stations_4326 = mrt_gdf_wo_lrt2.merge(train_stations_4326, how = "left", on = "STN_NAM_DE")

mrt_stations_4326

,CODE,STN_NAM_DE,LINE,line,COLOR,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry
0,NS1,JURONG EAST MRT STATION,North-South Line,NS,Red,1,0,None,None,MRT,POINT (103.74226 1.33321)
1,NS2,BUKIT BATOK MRT STATION,North-South Line,NS,Red,2,0,None,None,MRT,POINT (103.74954 1.349)
2,NS3,BUKIT GOMBAK MRT STATION,North-South Line,NS,Red,3,0,None,None,MRT,POINT (103.75191 1.35867)
3,NS4,CHOA CHU KANG MRT STATION,North-South Line,NS,Red,4,0,None,None,MRT,POINT (103.74437 1.38517)
4,NS5,YEW TEE MRT STATION,North-South Line,NS,Red,5,0,None,None,MRT,POINT (103.7474 1.39755)
...,...,...,...,...,...,...,...,...,...,...,...
215,TE26,MARINE PARADE MRT STATION,Thomson-East Coast Line,TE,Brown,26,0,None,None,MRT,POINT (103.90551 1.30287)
216,TE27,MARINE TERRACE MRT STATION,Thomson-East Coast Line,TE,Brown,27,0,None,None,MRT,POINT (103.91532 1.30679)
217,TE28,SIGLAP MRT STATION,Thomson-East Coast Line,TE,Brown,28,0,None,None,MRT,POINT (103.92988 1.30988)
218,TE29,BAYSHORE MRT STATION,Thomson-East Coast Line,TE,Brown,29,0,None,None,MRT,POINT (103.94344 1.31348)


In [128]:
# Group by 'grouper' and create LineString geometries
def create_linestring(group):
    # Check if there are at least two points to create a LineString
    if len(group) > 1:
        return LineString(group.to_list())
    else:
        # Return the original geometry if not enough points for a LineString
        return group.iloc[0]

# Apply the function to create LineString geometries
test_df_3857 = mrt_stations_3857.groupby(["line", "COLOR"])["geometry"].apply(create_linestring)

# Convert the resulting Series to a GeoDataFrame
geo_df_3857 = gpd.GeoDataFrame(test_df_3857, geometry="geometry").reset_index()

lines_gdf_3857 = geo_df_3857[geo_df_3857.geometry.type == "LineString"]
lines_gdf_3857 = lines_gdf_3857.reset_index()
lines_gdf_3857

,index,line,COLOR,geometry
0,0,CC,Orange,"LINESTRING (11560065.268 144621.33, 11560096.7..."
1,1,DT,Blue,"LINESTRING (11550685.441 153542.239, 11551033...."
2,2,EW,Green,"LINESTRING (11571578.429 152850.932, 11571122...."
3,3,NE,Purple,"LINESTRING (11557351.029 140881.522, 11557363...."
4,4,NS,Red,"LINESTRING (11548535.917 148425.544, 11549346...."
5,5,TE,Brown,"LINESTRING (11553368.462 161246.769, 11553422...."


In [129]:
# Apply the function to create LineString geometries
test_df_4326 = mrt_stations_4326.groupby(["line", "COLOR"])["geometry"].apply(create_linestring)

# Convert the resulting Series to a GeoDataFrame
geo_df_4326 = gpd.GeoDataFrame(test_df_4326, geometry="geometry").reset_index()

lines_gdf_4326 = geo_df_4326[geo_df_4326.geometry.type == "LineString"]
lines_gdf_4326 = lines_gdf_4326.reset_index()
lines_gdf_4326

,index,line,COLOR,geometry
0,0,CC,Orange,"LINESTRING (103.84583 1.29904, 103.84612 1.298..."
1,1,DT,Blue,"LINESTRING (103.76157 1.37916, 103.7647 1.3693..."
2,2,EW,Green,"LINESTRING (103.94926 1.37295, 103.94516 1.353..."
3,3,NE,Purple,"LINESTRING (103.82145 1.26546, 103.82157 1.265..."
4,4,NS,Red,"LINESTRING (103.74226 1.33321, 103.74954 1.349..."
5,5,TE,Brown,"LINESTRING (103.78567 1.44835, 103.78616 1.436..."


In [130]:
color_dict = {
    "Red": "#c03731",
    "Green": "#1f844d",
    "Purple": "#953aa6",
    "Orange": "#fba01d",
    "Blue": "#134f9a",
    "Brown": "#9d6633"
}

In [91]:
train_stations_4326.loc[166].geometry.is_empty

True

In [133]:
m = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

for index, row in lines_gdf_4326.iterrows():
    color = color_dict[row['COLOR']]
    locations = [(coord[1], coord[0]) for coord in row["geometry"].coords]
    folium.PolyLine(
        locations=locations,
        color=color,
        weight=2,
    ).add_to(m)


for index, row in mrt_stations_4326.iterrows():
    color = color_dict[row["COLOR"]]
    tooltip = f"""
    <div style="color: white; background-color: {color}; padding: 5px; border-radius: 5px;">
        <b>{row['CODE']}: {row['STN_NAM_DE']} {'MRT'} Station</b>
    </div>
    """
    radius=3
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=radius,
        weight=0.5,
        color=color,
        fill_color=color,
        fill_opacity=1,
        tooltip=tooltip,
    ).add_to(m)


In [134]:
m

In [126]:
mrt_stations_4326[mrt_stations_4326["STN_NAM_DE"] == "HILLVIEW MRT STATION"]

,CODE,STN_NAM_DE,LINE,line,COLOR,Order,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,geometry
140,DT3,HILLVIEW MRT STATION,Downtown Line,DT,Blue,3,0,None,None,MRT,POINT (103.76743 1.36234)


In [102]:
train_stations_4326.head()

,TYP_CD,STN_NAM,ATTACHEMEN,TYP_CD_DES,STN_NAM_DE,geometry
0,0,None,None,MRT,GALI BATU DEPOT,POINT (103.75609 1.39759)
1,0,None,None,MRT,HILLVIEW MRT STATION,POINT (103.76743 1.36234)
2,0,None,None,MRT,BEAUTY WORLD MRT STATION,POINT (103.77582 1.3412)
3,0,None,None,MRT,HUME MRT STATION,POINT (103.7691 1.35451)
4,0,None,None,MRT,BUKIT PANJANG MRT STATION,POINT (103.76157 1.37916)
